# Exploratory Analysis Sample IAV Vessel Snapshot

Exploration of the vessel snapshot sample AIS dateset provided by Volpe representing vessels within IAV/RAMP.

In [ ]:
file = '../data/raw/volpe-sample-iav-vessel-snapshot.csv'

In [ ]:
out_png_path = '../img/0_exploratory_analysis_iav_vessels_'

In [ ]:
out_html_path = '../files/0_exploratory_analysis_iav_vessels_'

## volpe-sample-iav-vessel-snapshot.csv

In [ ]:
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
import random

from shapely.geometry import Point

In [ ]:
!head '-n 5' $file

In [ ]:
# !sed '68628!d' $file

Some of the lines have bad data, skip them.

In [ ]:
df = pd.read_csv(file, sep=',', error_bad_lines=False)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

### GeoPandas and Folium

https://geopandas.org/gallery/plotting_with_folium.html  
https://jingwen-z.github.io/how-to-draw-a-variety-of-maps-with-folium-in-python/  

In [ ]:
geometry = geopandas.points_from_xy(df.lon, df.lat)

In [ ]:
geo_df = geopandas.GeoDataFrame(df, geometry=geometry)

In [ ]:
geo_df.head()

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
fig, ax = plt.subplots(figsize=(24,18))
world.plot(ax=ax, alpha=0.4, color='grey')
geo_df.plot(column='vesselType', ax=ax, legend=True)
plt.title('IAV Sample - AIS Messages by Vessel Type')

In [ ]:
fig.savefig(out_png_path + 'ais_global_coverage.png')

In [ ]:
geo_df['vesselType'].fillna('null', inplace=True)

In [ ]:
map = folium.Map(
    location = [32.316667,-64.833333],
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
    attr='Tiles &copy; Esri &mdash; EsriJapan, METI, Esri China (Hong Kong), and the GIS User Community',
    zoom_start=6
)

In [ ]:
# https://stackoverflow.com/questions/28999287/generate-random-colors-rgb
number_of_colors = geo_df.vesselType.unique().size

color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

In [ ]:
color_dict = {}

In [ ]:
vessel_types = geo_df.vesselType.unique()

In [ ]:
i=0
for v in vessel_types:
    color_dict[v] = {
        'vesselType': v,
        'color':color[i],
        'feature':folium.FeatureGroup(
                      name=v,
                      show=False
                  )
    }
    i+=1

In [ ]:
for i, v in geo_df.iterrows():
    
    t = v['vesselType']

    c = color_dict[t]['color']
   
    folium.CircleMarker(
        location=[v.geometry.y, v.geometry.x],
        radius=1,
        color=c,
        fill_color=c,
        fill=True
    ).add_to(color_dict[t]['feature'])

In [ ]:
for k, v in color_dict.items():
    v['feature'].add_to(map)

folium.LayerControl(
    name='Vessel Type',
    collapsed=True
).add_to(map)

In [ ]:
map.save(out_html_path + 'folium_map.html')

In [ ]:
map